In [423]:
import pathlib
import pandas as pd
import numpy as np
from itertools import combinations
import os
from tqdm import tqdm

In [2]:
path = pathlib.Path().resolve()

# First dataset
`US_Yale_University_OP0001562727_NV_Top_15_by_County_VIN_Prefix_202212.txt`

In [48]:
car_data = pd.read_csv(path / "ignored-data" / "US_Yale_University_OP0001562727_NV_Top_15_by_County_VIN_Prefix_202212.txt", sep = '|', chunksize = 1000)

Test the number of chunks and therefore the number of records: There are over 32,500,000 records in the file

In [27]:
# i = 0
# 
# for chunk in car_data:
#     i+=1
#     if i % 100 ==0:
#         print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

Take, say, 500 rows and put them in a DataFrame

In [49]:
df = pd.DataFrame()

i = 0

for chunk in car_data:
    if i == 0:
        df = chunk
    elif i < 500:
        df = pd.concat([df, chunk])
    else:
        break
        
    if i % 100 == 0:
        print(i)
    
    i+=1

0
100
200
300
400


This contains all the states of the US

In [58]:
df.head()

,REPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,TRANSACTION_PRICE,VIN_PREFIX,COUNTY_NAME,STATE_ABBRV,VEH_COUNT
0,201904,ACURA,ILX,2019,NaN,19UDE2F74K,SEMINOLE,FL,1
1,201911,ACURA,ILX,2019,NaN,19UDE2F74K,MIDDLESEX,NJ,2
2,201910,ACURA,ILX,2019,NaN,19UDE2F70K,COOK,IL,4
3,202107,ACURA,ILX,2020,NaN,19UDE2F72L,NORFOLK,MA,2
4,202010,ACURA,ILX,2020,NaN,19UDE2F72L,MIDDLESEX,MA,2


From Jan 2021 to December 2022 at least

In [60]:
max_date = max(df["REPORT_YEAR_MONTH"])
min_date = min(df["REPORT_YEAR_MONTH"])

print(f"The data goes from {min_date} to {max_date}")

The data goes from 201801 to 202212


CT is not in the dataset - appears to be top 15 states by population

In [65]:
df["STATE_ABBRV"].unique()

array(['FL', 'NJ', 'IL', 'MA', 'CA', 'OH', 'PA', 'NY', 'TX', 'VA', 'NC',
       'GA', 'MI', 'OK', 'AZ'], dtype=object)

# Second dataset

In [66]:
car_data_2 = pd.read_csv(path / "ignored-data" / "US_Yale_University_OP0001562727_NV_MA_CT_By_Zip_202210.txt", sep = '|')

In [67]:
max_date = max(car_data_2["REPORT_YEAR_MONTH"])
min_date = min(car_data_2["REPORT_YEAR_MONTH"])

print(f"The data goes from {min_date} to {max_date}")

The data goes from 201801 to 202210


See states

In [68]:
car_data_2["STATE_ABBRV"].unique()

array(['CT', 'MA'], dtype=object)

How much of this is CT?

In [84]:
CT_rows = len(car_data_2["STATE_ABBRV"] == "CT")
MA_rows = len(car_data_2["STATE_ABBRV"] == "MA")

print(f"CT comprises {CT_rows} of {CT_rows + MA_rows} = %{(CT_rows/(CT_rows+MA_rows)):.3f} of the dataset")

CT comprises 1489598 of 2979196 = %0.500 of the dataset


50% of the dataset is unusual given different populations, number of counties, etc...

In [76]:
car_data_2[car_data_2["STATE_ABBRV"]=="CT"].head()

,REPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,ZIP_CODE,STATE_ABBRV,VEH_COUNT
0,201811,SUBARU,CROSSTREK,2019,6074,CT,4
1,201903,NISSAN,ROGUE,2019,6074,CT,4
2,201912,SUBARU,CROSSTREK,2020,6074,CT,8
3,201908,HONDA,CR-V,2019,6074,CT,9
4,201909,NISSAN,SENTRA,2019,6074,CT,3


So they have different numbers of zip codes

In [79]:
car_data_2[car_data_2["STATE_ABBRV"]=="CT"]["ZIP_CODE"].nunique()

351

In [80]:
car_data_2[car_data_2["STATE_ABBRV"]=="MA"]["ZIP_CODE"].nunique()

644

Different dates? No.

In [82]:
car_data_2[car_data_2["STATE_ABBRV"]=="CT"]["REPORT_YEAR_MONTH"].nunique()

58

In [83]:
car_data_2[car_data_2["STATE_ABBRV"]=="MA"]["REPORT_YEAR_MONTH"].nunique()

58

Different numbers of makes & models?

In [91]:
# Create merged row
car_data_2_makemodel = car_data_2.drop(["ZIP_CODE", "VEH_COUNT"], axis =1 )

In [93]:
makemodel = [f"{make}_{model}" for make, model in zip(car_data_2_makemodel["MAKE"], car_data_2_makemodel["MODEL"])]

In [94]:
car_data_2_makemodel["makemodel"] = makemodel

In [98]:
car_data_2_makemodel[car_data_2_makemodel["STATE_ABBRV"]=="CT"]["makemodel"].nunique()

553

In [99]:
car_data_2_makemodel[car_data_2_makemodel["STATE_ABBRV"]=="MA"]["makemodel"].nunique()

577

Big comparison

# Third dataset

In [3]:
ct_vin_data = pd.read_csv(path / "ignored-data" / "US_Yale_University_OP0001562727_NV_CT_VIN_Prefix_202212.txt", sep = '|')

In [50]:
ct_vin_data.head()

,everREPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,TRANSACTION_PRICE,ZIP_CODE,VIN_PREFIX,COUNTY_NAME,STATE_ABBRV,VEH_COUNT,YEAR
0,201909,HONDA,CR-V,2019,25750.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019
1,201902,HONDA,CIVIC,2019,17522.0,6512,2HGFC2F65K,NEW HAVEN,CT,1,2019
2,201909,KIA,FORTE,2019,NaN,6078,3KPF24AD2K,HARTFORD,CT,1,2019
3,201911,LEXUS,RX,2020,55356.0,6037,2T2YGMDA2L,HARTFORD,CT,1,2019
4,201907,HONDA,CR-V,2019,25769.0,6084,2HKRW6H32K,TOLLAND,CT,1,2019


In [6]:
max_date = max(ct_vin_data["everREPORT_YEAR_MONTH"])
min_date = min(ct_vin_data["everREPORT_YEAR_MONTH"])
print(f"The data goes from {min_date} to {max_date}")

The data goes from 201801 to 202212


In [7]:
len(ct_vin_data)

924477

Count how many cars sold in 2018

In [30]:
ct_vin_data["YEAR"] = ct_vin_data["everREPORT_YEAR_MONTH"].astype(str).str[:4]

In [33]:
total_2018 = sum(ct_vin_data[ct_vin_data["YEAR"]=="2018"]["VEH_COUNT"])
print(total_2018)

265185


This seems reasonable... though perhaps high

**Attempt matching to VIN**

In [299]:
nhtsa_cleaned  = pd.read_csv(path / "ignored-data" / "NHTSA_cleaned.csv")

C:\Users\rajan\AppData\Local\Temp\ipykernel_11836\1854640047.py:1: DtypeWarning: Columns (7,9,10,13,20,25,30,34,42,44,56,58,68,78,80,114,115,138,146) have mixed types. Specify dtype option on import or set low_memory=False.
  nhtsa_cleaned  = pd.read_csv(path / "ignored-data" / "NHTSA_cleaned.csv")


This is where the EV drive unit is stored

In [40]:
nhtsa_cleaned["EVDriveUnit"].unique()

array([nan, 'Single Motor', 'Dual Motor'], dtype=object)

And this is where the VINs are stored, with a check digit.

In [41]:
nhtsa_cleaned["VIN"].head()

0    19UYA416*3A
1    19UYA417*3A
2    19UYA424*3A
3    19UYA425*3A
4    19UYA426*3A
Name: VIN, dtype: object

Simplify this

In [45]:
nhtsa_cleaned_small = nhtsa_cleaned.loc[:, nhtsa_cleaned.columns.isin(['VIN', 'EVDriveUnit'])]

In [47]:
nhtsa_cleaned_small["EVDriveUnit"] = nhtsa_cleaned_small["EVDriveUnit"].fillna('non-ev')

C:\Users\rajan\AppData\Local\Temp\ipykernel_11836\2338233233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhtsa_cleaned_small["EVDriveUnit"] = nhtsa_cleaned_small["EVDriveUnit"].fillna('non-ev')


Make the nhtsa code ready for matching (we need to drop the final character)
**NOTE: THIS WILL LEAD TO DUPLICATES TECHNICALLY - WE NEED TO KEEP ONLY THE FIRST OCURRENCE OF THAT VIN**

In [129]:
pd.set_option('display.max_columns', None)

In [131]:
# So here the issue was that there was an error with the VIN, that was then corrected.
nhtsa_cleaned[nhtsa_cleaned["VIN"].str[0:10]=="2HKRW6H3*K"]

,vin_row_start,ABS,ActiveSafetySysNote,AdaptiveCruiseControl,AdaptiveDrivingBeam,AdaptiveHeadlights,AdditionalErrorText,AirBagLocCurtain,AirBagLocFront,AirBagLocKnee,AirBagLocSeatCushion,AirBagLocSide,AutoReverseSystem,AutomaticPedestrianAlertingSound,AxleConfiguration,Axles,BasePrice,BatteryA,BatteryA_to,BatteryCells,BatteryInfo,BatteryKWh,BatteryKWh_to,BatteryModules,BatteryPacks,BatteryType,BatteryV,BatteryV_to,BedLengthIN,BedType,BlindSpotIntervention,BlindSpotMon,BodyCabType,BodyClass,BrakeSystemDesc,BrakeSystemType,BusFloorConfigType,BusLength,BusType,CAN_AACN,CIB,CashForClunkers,ChargerLevel,ChargerPowerKW,CoolingType,CurbWeightLB,CustomMotorcycleType,DaytimeRunningLight,DestinationMarket,DisplacementCC,DisplacementCI,DisplacementL,Doors,DriveType,DriverAssist,DynamicBrakeSupport,EDR,ESC,EVDriveUnit,ElectrificationLevel,EngineConfiguration,EngineCycles,EngineCylinders,EngineHP,EngineHP_to,EngineKW,EngineManufacturer,EngineModel,EntertainmentSystem,ErrorCode,ErrorText,ForwardCollisionWarning,FuelInjectionType,FuelTypePrimary,FuelTypeSecondary,GCWR,GCWR_to,GVWR,GVWR_to,KeylessIgnition,LaneCenteringAssistance,LaneDepartureWarning,LaneKeepSystem,LowerBeamHeadlampLightSource,Make,MakeID,Manufacturer,ManufacturerId,Model,ModelID,ModelYear,MotorcycleChassisType,MotorcycleSuspensionType,NCSABodyType,NCSAMake,NCSAMapExcApprovedBy,NCSAMapExcApprovedOn,NCSAMappingException,NCSAModel,NCSANote,NonLandUse,Note,OtherBusInfo,OtherEngineInfo,OtherMotorcycleInfo,OtherRestraintSystemInfo,OtherTrailerInfo,ParkAssist,PedestrianAutomaticEmergencyBraking,PlantCity,PlantCompanyName,PlantCountry,PlantState,PossibleValues,Pretensioner,RearAutomaticEmergencyBraking,RearCrossTrafficAlert,RearVisibilitySystem,SAEAutomationLevel,SAEAutomationLevel_to,SeatBeltsAll,SeatRows,Seats,SemiautomaticHeadlampBeamSwitching,Series,Series2,SteeringLocation,SuggestedVIN,TPMS,TopSpeedMPH,TrackWidth,TractionControl,TrailerBodyType,TrailerLength,TrailerType,TransmissionSpeeds,TransmissionStyle,Trim,Trim2,Turbo,VIN,ValveTrainDesign,VehicleDescriptor,VehicleType,WheelBaseLong,WheelBaseShort,WheelBaseType,WheelSizeFront,WheelSizeRear,Wheels,Windows,CAid,EPAid
38636,84501,Standard,NaN,NaN,NaN,NaN,NaN,1st and 2nd Rows,1st Row (Driver and Passenger),NaN,NaN,1st Row (Driver and Passenger),Standard,Standard,NaN,2.0,25750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,NaN,NaN,Not Applicable,NaN,Not Applicable,NaN,NaN,NaN,NaN,NaN,Water,NaN,Not Applicable,Standard,NaN,2400.0,146.456986,2.4,5.0,4WD,NaN,Standard,NaN,Standard,NaN,NaN,In-Line,4.0,4.0,184.0,NaN,137.2088,Honda,K24W9/K24V9,NaN,6,6 - Incomplete VIN,NaN,NaN,Gasoline,NaN,NaN,NaN,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",NaN,Standard,NaN,NaN,NaN,NaN,HONDA,474.0,HONDA,990,CR V,1865.0,2019,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct Fuel Injection,NaN,"seat belts: front, rear, rear center",NaN,NaN,NaN,ALLISTON,NaN,CANADA,ONTARIO,NaN,NaN,NaN,NaN,Standard,NaN,NaN,Manual,2.0,5.0,Standard,LX,NaN,Left-Hand Drive (LHD),NaN,Direct,NaN,NaN,Standard,Not Applicable,NaN,Not Applicable,NaN,Continuously Variable Transmission (CVT),NaN,NaN,NaN,2HKRW6H3*KH,Dual Overhead Cam (DOHC),2HKRW6H3*KH,MULTIPURPOSE PASSENGER VEHICLE (MPV),NaN,104.7,NaN,17.0,17.0,4.0,NaN,30301.0,11237.0
38637,84501,Standard,NaN,NaN,NaN,NaN,"In the Possible values section, the Numeric va...",1st and 2nd Rows,1st Row (Driver and Passenger),NaN,NaN,1st Row (Driver and Passenger),Standard,Standard,NaN,2.0,25750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,NaN,NaN,Not Applicable,NaN,Not Applicable,NaN,NaN,NaN,NaN,NaN,Water,NaN,Not Applicable,Standard,NaN,2400.0,146.456986,2.4,5.0,4WD,NaN,Standard,NaN,Standard,NaN,NaN,In-Line,4.0,4.0,184.0,NaN,137.2088,Honda,K24W9/K24V9,NaN,"4,6,14","4 - VIN corrected, error in one position only ...",NaN,NaN,Gasoline,NaN,NaN,NaN,"Class 1C: 4,001 - 5,000 lb (1,81

In [143]:
nhtsa_cleaned[nhtsa_cleaned["VIN"].str[0:10]=="W1Y5EDHY*L"]

,vin_row_start,ABS,ActiveSafetySysNote,AdaptiveCruiseControl,AdaptiveDrivingBeam,AdaptiveHeadlights,AdditionalErrorText,AirBagLocCurtain,AirBagLocFront,AirBagLocKnee,AirBagLocSeatCushion,AirBagLocSide,AutoReverseSystem,AutomaticPedestrianAlertingSound,AxleConfiguration,Axles,BasePrice,BatteryA,BatteryA_to,BatteryCells,BatteryInfo,BatteryKWh,BatteryKWh_to,BatteryModules,BatteryPacks,BatteryType,BatteryV,BatteryV_to,BedLengthIN,BedType,BlindSpotIntervention,BlindSpotMon,BodyCabType,BodyClass,BrakeSystemDesc,BrakeSystemType,BusFloorConfigType,BusLength,BusType,CAN_AACN,CIB,CashForClunkers,ChargerLevel,ChargerPowerKW,CoolingType,CurbWeightLB,CustomMotorcycleType,DaytimeRunningLight,DestinationMarket,DisplacementCC,DisplacementCI,DisplacementL,Doors,DriveType,DriverAssist,DynamicBrakeSupport,EDR,ESC,EVDriveUnit,ElectrificationLevel,EngineConfiguration,EngineCycles,EngineCylinders,EngineHP,EngineHP_to,EngineKW,EngineManufacturer,EngineModel,EntertainmentSystem,ErrorCode,ErrorText,ForwardCollisionWarning,FuelInjectionType,FuelTypePrimary,FuelTypeSecondary,GCWR,GCWR_to,GVWR,GVWR_to,KeylessIgnition,LaneCenteringAssistance,LaneDepartureWarning,LaneKeepSystem,LowerBeamHeadlampLightSource,Make,MakeID,Manufacturer,ManufacturerId,Model,ModelID,ModelYear,MotorcycleChassisType,MotorcycleSuspensionType,NCSABodyType,NCSAMake,NCSAMapExcApprovedBy,NCSAMapExcApprovedOn,NCSAMappingException,NCSAModel,NCSANote,NonLandUse,Note,OtherBusInfo,OtherEngineInfo,OtherMotorcycleInfo,OtherRestraintSystemInfo,OtherTrailerInfo,ParkAssist,PedestrianAutomaticEmergencyBraking,PlantCity,PlantCompanyName,PlantCountry,PlantState,PossibleValues,Pretensioner,RearAutomaticEmergencyBraking,RearCrossTrafficAlert,RearVisibilitySystem,SAEAutomationLevel,SAEAutomationLevel_to,SeatBeltsAll,SeatRows,Seats,SemiautomaticHeadlampBeamSwitching,Series,Series2,SteeringLocation,SuggestedVIN,TPMS,TopSpeedMPH,TrackWidth,TractionControl,TrailerBodyType,TrailerLength,TrailerType,TransmissionSpeeds,TransmissionStyle,Trim,Trim2,Turbo,VIN,ValveTrainDesign,VehicleDescriptor,VehicleType,WheelBaseLong,WheelBaseShort,WheelBaseType,WheelSizeFront,WheelSizeRear,Wheels,Windows,CAid,EPAid
57012,124601,NaN,NaN,NaN,NaN,NaN,NaN,Driver Seat Only,1st Row (Driver and Passenger),NaN,NaN,1st Row (Driver and Passenger),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cargo Van,NaN,Hydraulic,Not Applicable,NaN,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,NaN,NaN,3000.0,183.071232,3.0,NaN,2WD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OM642,NaN,6,6 - Incomplete VIN,NaN,NaN,Diesel,NaN,NaN,NaN,"Class 1D: 5,001 - 6,000 lb (2,268 - 2,722 kg)",NaN,NaN,NaN,NaN,NaN,NaN,MERCEDES-BENZ,449.0,MERCEDES-BENZ,1023,Sprinter,1703.0,2020,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Body Length:7367 mm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHARLESTON,NaN,UNITED STATES (USA),SOUTH CAROLINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3500,907 (VS30),NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,NaN,Not Applicable,NaN,NaN,NaN,NaN,NaN,W1Y5EDHY*LT,NaN,W1Y5EDHY*LT,TRUCK,NaN,170.0,NaN,NaN,NaN,NaN,NaN,31620.0,NaN
57013,124601,NaN,NaN,NaN,NaN,NaN,NaN,Driver Seat Only,1st Row (Driver and Passenger),NaN,NaN,1st Row (Driver and Passenger),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cargo Van,NaN,Hydraulic,Not Applicable,NaN,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,NaN,NaN,3000.0,183.071232,3.0,NaN,2WD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OM642,NaN,6,6 - Incomplete VIN,NaN,NaN,Diesel,NaN,NaN,NaN,"Class 1D: 5,001 - 6,000 lb (2,268 - 2,722 kg)",NaN,NaN,NaN,NaN,NaN,NaN,MERCEDES-BENZ,449.0,MERCEDES-BENZ,1023,Sprinter,1703.0,2020,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Body Length:7367 mm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DÜSSELDORF,NaN,GERMANY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3500,907 (VS30),NaN,NaN,NaN,NaN,NaN,NaN,Not Applicable,NaN,Not Applicable,NaN,NaN,NaN,NaN,NaN,W1Y5EDHY*LP,NaN,W1

In [56]:
nhtsa_cleaned_small["VIN"] = nhtsa_cleaned_small["VIN"].str[0:10]

C:\Users\rajan\AppData\Local\Temp\ipykernel_11836\1006857570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhtsa_cleaned_small["VIN"] = nhtsa_cleaned_small["VIN"].str[0:10]


Make the RL Polk data ready for matching (by converting the check character to a "*"

In [64]:
ct_vin_data.head()

,everREPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,TRANSACTION_PRICE,ZIP_CODE,VIN_PREFIX,COUNTY_NAME,STATE_ABBRV,VEH_COUNT,YEAR
0,201909,HONDA,CR-V,2019,25750.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019
1,201902,HONDA,CIVIC,2019,17522.0,6512,2HGFC2F65K,NEW HAVEN,CT,1,2019
2,201909,KIA,FORTE,2019,NaN,6078,3KPF24AD2K,HARTFORD,CT,1,2019
3,201911,LEXUS,RX,2020,55356.0,6037,2T2YGMDA2L,HARTFORD,CT,1,2019
4,201907,HONDA,CR-V,2019,25769.0,6084,2HKRW6H32K,TOLLAND,CT,1,2019


In [67]:
ct_vin_data["VIN_PREFIX_NEW"] = ct_vin_data["VIN_PREFIX"].str[0:8]+"*"+ct_vin_data["VIN_PREFIX"].str[9:10]

***
# Identify duplicate VINS in NHTSA dataset
**Note:**
* NHTSA uses the first 11 alpha-numerics with the check variable in the 9th place replaced by '\*': ie ABCEDFGH\*JK
* RL Polk just uses the first 10 alpha-numerics including the check variable. ie ABCEDFGHIJ
* Therefore, to merge on the NHTSA VIN dataset, we must first **drop the 11th alpha-numeric**. However, this **leads to duplicates in the NHTSA VIN list** since VINs that were previously separate now become identical.

To address this, we undertake a quality check to determine how much 10-long VINS within the same 11-long family differ from one another. If they do not differ on meaningful characteristics, then we can arbitrarily drop the 11th character and simply choose the first 10-long VIN matching that pattern

### Inspect those VINS that are not unique

In [363]:
# Get those rows that are not unique

# Create a copy of the initial list
nhtsa_cleaned_notunique = nhtsa_cleaned.copy(deep = True)

# Drop the 11th character of the VINs in the first list
nhtsa_cleaned_notunique["VIN"] = nhtsa_cleaned_notunique["VIN"].str[0:10]

# Create counter to sum
nhtsa_cleaned_notunique["counter"] = 1

# Group by VIN and take a count. The count of the column "counter" reflects the number of repeats
nhtsa_cleaned_notunique_grouped = nhtsa_cleaned_notunique.groupby("VIN").count().reset_index()

In [369]:
# View result
nhtsa_cleaned_notunique_grouped.head()

,VIN,vin_row_start,ABS,ActiveSafetySysNote,AdaptiveCruiseControl,AdaptiveDrivingBeam,AdaptiveHeadlights,AdditionalErrorText,AirBagLocCurtain,AirBagLocFront,AirBagLocKnee,AirBagLocSeatCushion,AirBagLocSide,AutoReverseSystem,AutomaticPedestrianAlertingSound,AxleConfiguration,Axles,BasePrice,BatteryA,BatteryA_to,BatteryCells,BatteryInfo,BatteryKWh,BatteryKWh_to,BatteryModules,BatteryPacks,BatteryType,BatteryV,BatteryV_to,BedLengthIN,BedType,BlindSpotIntervention,BlindSpotMon,BodyCabType,BodyClass,BrakeSystemDesc,BrakeSystemType,BusFloorConfigType,BusLength,BusType,CAN_AACN,CIB,CashForClunkers,ChargerLevel,ChargerPowerKW,CoolingType,CurbWeightLB,CustomMotorcycleType,DaytimeRunningLight,DestinationMarket,DisplacementCC,DisplacementCI,DisplacementL,Doors,DriveType,DriverAssist,DynamicBrakeSupport,EDR,ESC,EVDriveUnit,ElectrificationLevel,EngineConfiguration,EngineCycles,EngineCylinders,EngineHP,EngineHP_to,EngineKW,EngineManufacturer,EngineModel,EntertainmentSystem,ErrorCode,ErrorText,ForwardCollisionWarning,FuelInjectionType,FuelTypePrimary,FuelTypeSecondary,GCWR,GCWR_to,GVWR,GVWR_to,KeylessIgnition,LaneCenteringAssistance,LaneDepartureWarning,LaneKeepSystem,LowerBeamHeadlampLightSource,Make,MakeID,Manufacturer,ManufacturerId,Model,ModelID,ModelYear,MotorcycleChassisType,MotorcycleSuspensionType,NCSABodyType,NCSAMake,NCSAMapExcApprovedBy,NCSAMapExcApprovedOn,NCSAMappingException,NCSAModel,NCSANote,NonLandUse,Note,OtherBusInfo,OtherEngineInfo,OtherMotorcycleInfo,OtherRestraintSystemInfo,OtherTrailerInfo,ParkAssist,PedestrianAutomaticEmergencyBraking,PlantCity,PlantCompanyName,PlantCountry,PlantState,PossibleValues,Pretensioner,RearAutomaticEmergencyBraking,RearCrossTrafficAlert,RearVisibilitySystem,SAEAutomationLevel,SAEAutomationLevel_to,SeatBeltsAll,SeatRows,Seats,SemiautomaticHeadlampBeamSwitching,Series,Series2,SteeringLocation,SuggestedVIN,TPMS,TopSpeedMPH,TrackWidth,TractionControl,TrailerBodyType,TrailerLength,TrailerType,TransmissionSpeeds,TransmissionStyle,Trim,Trim2,Turbo,ValveTrainDesign,VehicleDescriptor,VehicleType,WheelBaseLong,WheelBaseShort,WheelBaseType,WheelSizeFront,WheelSizeRear,Wheels,Windows,CAid,EPAid,counter
0,19UDE2F3*G,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,1,1,1,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1
1,19UDE2F3*H,3,3,3,0,3,0,1,3,3,0,0,3,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,3,3,0,0,3,0,3,0,0,0,0,0,3,0,3,3,0,3,3,3,3,3,0,3,0,3,0,0,3,3,3,3,0,3,3,3,0,3,3,0,0,3,0,0,0,3,0,3,0,0,0,0,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,2,0,2,2,1,0,0,0,3,0,0,3,3,3,3,3,0,3,1,3,3,0,3,3,0,3,3,3,0,0,0,3,3,3,0,3,0,3,3,3,0,3,3,3
2,19UDE2F3*J,2,2,2,2,0,0,1,2,2,0,0,2,2,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,2,0,0,2,0,2,0,2,0,0,0,2,0,2,2,0,2,2,2,2,2,0,2,0,2,0,0,2,2,2,2,0,2,2,2,0,2,2,0,0,2,0,0,0,2,0,2,0,0,0,0,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,1,0,1,1,1,0,0,0,2,0,0,2,2,2,2,2,0,2,1,2,2,0,2,2,0,2,2,2,0,0,0,2,2,2,0,2,0,2,2,2,0,2,2,2
3,19UDE2F3*K,1,1,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,0,1,1,1,1,1,0,1,0,1,0,0,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1,0,1,0,1,0,0,1,1,0,1,1,1,0,0,0,1,1,1,0,1,0,1,1,1,0,1,1,1
4,19UDE2F3*L,7,7,0,0,0,0,0,7,7,0,0,7,7,0,0,7,7,0,0,0,0,0,0,0,0,0,0,0,0,7,0,7,7,7,0,0,7,0,7,7,0,0,0,0,7,0,7,7,0,7,7,7,7,7,0,7,7,7,0,0,7,7,7,7,0,7,7,7,0,7,7,0,0,7,0,0,0,7,0,0,0,0,0,0,7,7,7,7,7,7,7,7,7,0,0,0,0,0,0,0,0,7,0,7,0,7,0,0,0,7,0,7,7,0,0,0,0,7,0,0,7,7,7,7,7,0,7,0,7,0,0,7,7,0,7,7,7,0,0,0,7,7,7,0,7,0,0,0,7,0,7,0,7


In [370]:
# See how long this is
print(f"The length of the grouped dataset is {len(nhtsa_cleaned_notunique_grouped)}")

The length of the grouped dataset is 48645


In [ ]:
# Create a list of the nonunique VINs

# To do this, we take all of the VINs with a counter of greater than 1.
# This is the list of 10-digit VINs, belonging to an 11-digit family, that has more than 1 member (i.e.;
# 10-digit VINs that have siblings)
vins_not_unique = list(nhtsa_cleaned_notunique_grouped[nhtsa_cleaned_notunique_grouped["counter"]>1]["VIN"])

In [371]:
# Look to see how many of these VINs there are
len(vins_not_unique)

12590

In [372]:
# Create a subset of the initial DF, containing only non-unique VINs
nhtsa_cleaned_notunique_only = nhtsa_cleaned_notunique.loc[nhtsa_cleaned_notunique.VIN.isin(vins_not_unique), :]
len(nhtsa_cleaned_notunique_only)

36081

### Explanation
We have cleaned the NHTSA dataframe to contain only entries for 10-digit VINs, belonging to multi-member 11-digit families. That is, when we shorten from an 11-digit to a 10-digit VIN by dropping the 11th digit, these are the VINs that now look identical to some other VIN.

We now investigate how similar the vehicle attributes are for these VINs

### Log differences between identical VINs

In [373]:
# Replace all NaNs with 0 to facilitate comparison later on
nhtsa_cleaned_notunique_only = nhtsa_cleaned_notunique_only.replace(np.nan,0)

In [367]:
# Create a file to record progress
outfh_path = path / "data" / "outputs" / "track_diffs.txt"
out_fh = open(outfh_path, "w" if not os.path.exists(outfh_path) else "a")

In [ ]:
# This function will output a large text file, containing all the differences between
# Rows that share the same VIN

def diffs(df):
    index = 0
    num_indices = len(vins_not_unique)
    # Do this for every VIN in the non unique vins list 
    for vin in vins_not_unique:
        if (index % 500 == 0):
            print(f"Working on index number {index} of {num_indices}")
        
        # Extract the entries of the dataframe with that VIN
        entries = df[df["VIN"] == vin].reset_index()
        num_entries = len(entries)
        
        # Record it in the log file
        out_fh.write(f"Working on VIN number {vin}, number of entries: {num_entries}\n")
        
        # Only if that VIN is to be found
        if num_entries > 0:
            
            # Create a list of all possible combinations of rows
            combos = list(combinations(list(np.arange(0, num_entries)), 2))
            
            # Go column by column
            columns = list(entries.columns)
            columns.remove("index")
            columns.remove("vin_row_start")
            
            for column in columns:
                diff_count = 0
                cells = list(entries[column])
                
                # Compare combos
                for combo in combos:
                    if (cells[combo[0]] != cells[combo[1]]):
                        out_fh.write(f"... Difference in column {column}:\n")
                        out_fh.write(f"... ... Difference between {cells[combo[0]]} and {cells[combo[1]]}\n")
                        
            index +=1
    out_fh.close()

diffs(nhtsa_cleaned_notunique_only)

This text file shows that the majority of the differences between vehicles that share the same shortened 10-digit VIN  are aesthetic only. That is, they pertain mostly to the following columns:
* AdditionalErrorText
* ErrorCode
* ErrorText
* PlantState
* PlantCounty
* VehicleDescriptor

These differences mean we should be able to freely drop the 11th digit, and simply match on the 10 remaining (we can choose the first row for which that 10-digit VIN appears).

### Produce a shortened VIN list

In [415]:
# Create a new DF
nhtsa_for_merge = nhtsa_cleaned.copy(deep = True)

# Set VINs to 10-long
nhtsa_for_merge["VIN"] = nhtsa_cleaned["VIN"].str[0:10]

# Those with drop set to 0 are to be kept
nhtsa_for_merge["drop"] = 0

# Sort by VIN so we can check the row above
nhtsa_for_merge = nhtsa_for_merge.sort_values("VIN").reset_index()

In [419]:
# Loop through the rows and find if the VIN is the same as the VIN above
# If it is, marke it to be dropped
for index, row in tqdm(nhtsa_for_merge.iterrows()):
    if nhtsa_for_merge.iloc[index, nhtsa_for_merge.columns.get_loc("VIN")] == nhtsa_for_merge.iloc[index - 1,  nhtsa_for_merge.columns.get_loc("VIN")]:
        nhtsa_for_merge.iloc[index, nhtsa_for_merge.columns.get_loc("drop")] = 1

72136it [00:09, 7621.61it/s]


In [422]:
nhtsa_for_merge[nhtsa_for_merge["drop"]==0]["VIN"].nunique()

48645

In [424]:
# Drop all marked as 1
nhtsa_for_merge = nhtsa_for_merge[nhtsa_for_merge["drop"]==0]

In [ ]:
# Prepare the EV drive unit
nhtsa_for_merge["EVDriveUnit"] = nhtsa_for_merge["EVDriveUnit"].fillna('non-ev')

# Drop unnecessary columns
nhtsa_for_merge = nhtsa_for_merge.iloc[:, nhtsa_for_merge.columns.isin(["VIN", "EVDriveUnit"])]

***

**Attempt match**

In [104]:
ct_vin_data_to_match = ct_vin_data.loc[:, ~ct_vin_data.columns.isin(["VIN_PREFIX", "TRANSACTION_PRICE"])]

In [102]:
ct_vin_data_to_match.head()

,everREPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,TRANSACTION_PRICE,ZIP_CODE,COUNTY_NAME,STATE_ABBRV,VEH_COUNT,YEAR,VIN_PREFIX_NEW
0,201909,HONDA,CR-V,2019,25750.0,6461,NEW HAVEN,CT,1,2019,2HKRW6H3*K
1,201902,HONDA,CIVIC,2019,17522.0,6512,NEW HAVEN,CT,1,2019,2HGFC2F6*K
2,201909,KIA,FORTE,2019,NaN,6078,HARTFORD,CT,1,2019,3KPF24AD*K
3,201911,LEXUS,RX,2020,55356.0,6037,HARTFORD,CT,1,2019,2T2YGMDA*L
4,201907,HONDA,CR-V,2019,25769.0,6084,TOLLAND,CT,1,2019,2HKRW6H3*K


In [97]:
len(nhtsa_cleaned_small)

72136

In [112]:
ct_vin_data[(ct_vin_data["VIN_PREFIX"]=="2HKRW6H39K") & (ct_vin_data["everREPORT_YEAR_MONTH"]==201909) ]

,everREPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,TRANSACTION_PRICE,ZIP_CODE,VIN_PREFIX,COUNTY_NAME,STATE_ABBRV,VEH_COUNT,YEAR,VIN_PREFIX_NEW
0,201909,HONDA,CR-V,2019,25750.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019,2HKRW6H3*K
186739,201909,HONDA,CR-V,2019,26448.0,6784,2HKRW6H39K,FAIRFIELD,CT,1,2019,2HKRW6H3*K
358514,201909,HONDA,CR-V,2019,24273.0,6051,2HKRW6H39K,HARTFORD,CT,1,2019,2HKRW6H3*K
692923,201909,HONDA,CR-V,2019,17653.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019,2HKRW6H3*K
762152,201909,HONDA,CR-V,2019,NaN,6051,2HKRW6H39K,HARTFORD,CT,1,2019,2HKRW6H3*K


In [435]:
len(ct_vin_data)

924477

In [433]:
match_2 = ct_vin_data.merge(nhtsa_for_merge, left_on = "VIN_PREFIX_NEW", right_on = "VIN")

In [115]:
match = ct_vin_data.merge(nhtsa_cleaned_small, left_on = "VIN_PREFIX_NEW", right_on = "VIN")

In [122]:
nhtsa_cleaned_small[nhtsa_cleaned_small["VIN"]=="2HKRW6H3*K"]

,EVDriveUnit,VIN
38636,non-ev,2HKRW6H3*K
38637,non-ev,2HKRW6H3*K


In [116]:
match[(match["VIN_PREFIX"]=="2HKRW6H39K") & (match["everREPORT_YEAR_MONTH"]==201909) ]

,everREPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,TRANSACTION_PRICE,ZIP_CODE,VIN_PREFIX,COUNTY_NAME,STATE_ABBRV,VEH_COUNT,YEAR,VIN_PREFIX_NEW,EVDriveUnit,VIN
0,201909,HONDA,CR-V,2019,25750.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
1,201909,HONDA,CR-V,2019,25750.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
490,201909,HONDA,CR-V,2019,26448.0,6784,2HKRW6H39K,FAIRFIELD,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
491,201909,HONDA,CR-V,2019,26448.0,6784,2HKRW6H39K,FAIRFIELD,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
958,201909,HONDA,CR-V,2019,24273.0,6051,2HKRW6H39K,HARTFORD,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
959,201909,HONDA,CR-V,2019,24273.0,6051,2HKRW6H39K,HARTFORD,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
1764,201909,HONDA,CR-V,2019,17653.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
1765,201909,HONDA,CR-V,2019,17653.0,6461,2HKRW6H39K,NEW HAVEN,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
1902,201909,HONDA,CR-V,2019,NaN,6051,2HKRW6H39K,HARTFORD,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
1903,201909,HONDA,CR-V,2019,NaN,6051,2HKRW6H39K,HARTFORD,CT,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K


In [95]:
len(ct_vin_data)

924477

In [93]:
len(match)

1641585

In [84]:
match.columns

Index(['everREPORT_YEAR_MONTH', 'MAKE', 'MODEL', 'MODEL_YEAR',
       'TRANSACTION_PRICE', 'ZIP_CODE', 'VIN_PREFIX', 'COUNTY_NAME',
       'STATE_ABBRV', 'VEH_COUNT', 'YEAR', 'VIN_PREFIX_NEW', 'EVDriveUnit',
       'VIN'],
      dtype='object')

In [436]:
evs_sold = match_2[match_2["EVDriveUnit"]!="non-ev"]

In [437]:
evs_sold["VEH_COUNT"].sum()

1637

In [78]:
evs_sold[evs_sold["YEAR"]=="2022"]["VEH_COUNT"].sum()

25666

In [85]:
match_short = match.loc[:, match.columns.isin(["everREPORT_YEAR_MONTH", "MAKE", "MODEL", "MODEL_YEAR", "ZIP_CODE", "VIN_PREFIX", "COUNTY_NAME", "VEH_COUNT", "YEAR", 'VIN_PREFIX_NEW', 'EVDriveUnit',
       'VIN'])]

In [86]:
match_short

,everREPORT_YEAR_MONTH,MAKE,MODEL,MODEL_YEAR,ZIP_CODE,VIN_PREFIX,COUNTY_NAME,VEH_COUNT,YEAR,VIN_PREFIX_NEW,EVDriveUnit,VIN
0,201909,HONDA,CR-V,2019,6461,2HKRW6H39K,NEW HAVEN,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
1,201909,HONDA,CR-V,2019,6461,2HKRW6H39K,NEW HAVEN,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
2,201907,HONDA,CR-V,2019,6084,2HKRW6H32K,TOLLAND,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
3,201907,HONDA,CR-V,2019,6084,2HKRW6H32K,TOLLAND,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
4,201910,HONDA,CR-V,2019,6611,2HKRW6H33K,FAIRFIELD,1,2019,2HKRW6H3*K,non-ev,2HKRW6H3*K
...,...,...,...,...,...,...,...,...,...,...,...,...
1641580,202010,MERCEDES-BENZ,SPRINTER,2020,6078,W1Y5EDHY6L,HARTFORD,1,2020,W1Y5EDHY*L,non-ev,W1Y5EDHY*L
1641581,202010,MERCEDES-BENZ,SPRINTER,2020,6078,W1Y5EDHY6L,HARTFORD,1,2020,W1Y5EDHY*L,non-ev,W1Y5EDHY*L
1641582,202001,RAM,RAM,2020,6002,3C6UR5JJ4L,HARTFORD,1,2020,3C6UR5JJ*L,non-ev,3C6UR5JJ*L
1641583,202010,KIA,SORENTO,2019,6811,5XYPK4A50K,FAIRFIELD,1,2020,5XYPK4A5*K,non-ev,5XYPK4A5*K
